In [2]:
pip install implicit

In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [178]:
data = pd.read_csv('../Рекомендательные системы/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [179]:
data['week_no'].nunique()

95

In [180]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [186]:
data['revenue']=data['quantity']*data['sales_value']

In [187]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [188]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]


In [202]:
item_weights=data.groupby('item_id')['revenue'].sum().reset_index()
#item_weights.head()
total=item_weights['revenue'].sum()
item_weights['weights']=item_weights['revenue']/total
item_weights.head()

,item_id,revenue,weights
0,25671,62.82,8.282896e-09
1,26081,0.99,1.305327e-10
2,26093,1.59,2.096435e-10
3,26190,1.54,2.030509e-10
4,26355,3.96,5.221310e-10


In [210]:
items_weights=item_weights.drop(['revenue'], axis=1)
items_weights.head()

,item_id,weights
0,25671,8.282896e-09
1,26081,1.305327e-10
2,26093,2.096435e-10
3,26190,2.030509e-10
4,26355,5.221310e-10


In [229]:
import random

def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_weights['item_id'])
    w=np.array(items_weights['weights'])
    recs = np.random.choice(items, p=w,size=n, replace=False)
    
    return recs.tolist()

In [230]:
%%time

#items = data_train.item_id.unique()

#items_weights['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
weighted_random_recommendation(items_weights)

CPU times: total: 46.9 ms
Wall time: 133 ms


[6534178, 6534166, 545926, 6533889, 397896]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [131]:
def weighted_random_recommendation(items,items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_coder
    
    items = np.array(items)
    items_weights=np.array(items_weights)
    recs = np.random.choice(items, p=items_weights, size=n, replace=False)
    
    return recs.tolist()

In [135]:
%%time

items = data_train.item_id.unique()
type(items)

resul['random_recommendation'] = resul['user_id'].apply(lambda x: weighted_random_recommendation(items, n=5))
resul.head(2)

TypeError: weighted_random_recommendation() missing 1 required positional argument: 'items_weights'

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [121]:
result = pd.read_csv('../Рекомендательные системы/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [48]:
res=result.drop(['user_id', 'actual'], axis=1).iloc[:5, :]
res

,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,"[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,"[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]"
4,"[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [102]:
res['own_purchases'][0]


'[999999, 1082185, 1029743, 995785, 1004906]'

In [110]:
import ast

In [113]:
m = ast.literal_eval(res['own_purchases'][0])
type(m[0])

int

In [109]:
#b = res['own_purchases'][0].strip('][').split(', ')
#type(b[0])

str

In [78]:
res.columns[:-1]
    

Index(['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine',
       'tfidf'],
      dtype='object')

In [103]:
res.iloc[:,-1][0]

'[999999, 1082185, 1029743, 995785, 1004906]'

In [50]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [117]:
final={}
for x in res.columns[:-1]:
    f=[]
    for u,v in zip(res[x],res.iloc[:,-1]):
        u=ast.literal_eval(u)
        v=ast.literal_eval(v)
        f.append(precision_at_k(u,v))
    final[x]=max(f)
    
final

{'random_recommendation': 0.0,
 'popular_recommendation': 0.6,
 'itemitem': 0.0,
 'cosine': 0.0,
 'tfidf': 0.0}

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code